In [2]:
source("charger.R")
mondata <- charger(2105482)

In [3]:
mondata

,Sales,Price,Advertising,Region
,<dbl>,<int>,<int>,<int>
150,4.67,111,0,0
68,12.01,94,0,1
273,9.40,96,17,0
43,5.05,117,0,1
5,6.42,126,5,1
218,8.47,112,0,0
159,10.43,24,0,1
74,11.62,139,4,1
209,6.98,97,0,0
